## GOAL
Verify Qiskit and IBM Runtime are installed.


In [2]:
# Import Qiskit and IBM Runtime.
import qiskit
# import qiskit_ibm_runtime

# Print versions to confirm installation.
print("qiskit version:", qiskit.__version__)
# print("qiskit-ibm-runtime version:", qiskit_ibm_runtime.__version__)


qiskit version: 1.4.2


## EXERCISE
If imports fail, confirm you selected the .venv interpreter in VS Code.


## GOAL
Verify your IBM token is available as an environment variable.


In [ ]:
# Read environment variables.
import os

# Get token from environment.
token = os.getenv("QISKIT_IBM_TOKEN")
if token:
    print("Token found:", token[:3] + "***")
else:
    print("Token not found. Set QISKIT_IBM_TOKEN and restart the kernel.")


## EXERCISE
Set QISKIT_IBM_TOKEN in your .env or terminal and re-run this cell.


## GOAL
Initialize QiskitRuntimeService (no hardware job yet).


In [ ]:
# Import the runtime service.
from qiskit_ibm_runtime import QiskitRuntimeService

# Stop early if the token is missing.
if not token:
    raise SystemExit("Token missing. Set QISKIT_IBM_TOKEN and restart the kernel.")

# Create the service using the token.
service = QiskitRuntimeService(channel="ibm_quantum", token=token)

# List available backends as a basic connectivity check.
backends = service.backends()
print("Backends available:", len(backends))


## EXERCISE
Confirm that the backend list is not empty.


## GOAL
Understand |0>, |1>, |+> on the Bloch sphere with minimal math.


In [ ]:
# Import tools for circuits and visualization.
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector

# Build |0> state.
qc0 = QuantumCircuit(1)
state0 = Statevector.from_instruction(qc0)
plot_bloch_multivector(state0)

# Build |1> state using X gate.
qc1 = QuantumCircuit(1)
qc1.x(0)
state1 = Statevector.from_instruction(qc1)
plot_bloch_multivector(state1)

# Build |+> state using H gate.
qcp = QuantumCircuit(1)
qcp.h(0)
statep = Statevector.from_instruction(qcp)
plot_bloch_multivector(statep)


## EXERCISE
Create |-> by applying X then H to |0>. Predict where it appears on the Bloch sphere.


## GOAL
Measure a 1-qubit circuit with shots and read counts.


In [ ]:
# Import the sampler and histogram.
from qiskit.primitives import Sampler
from qiskit.visualization import plot_histogram

# Build a 1-qubit circuit and measure.
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.measure(0, 0)

# Run on local sampler.
sampler = Sampler()
result = sampler.run([qc], shots=1024).result()
quasi = result.quasi_dists[0]
counts = {k: int(v * 1024) for k, v in quasi.items()}
print("Counts:", counts)
plot_histogram(counts)


## EXERCISE
Change shots to 4096 and compare the stability of the counts.


## GOAL
Create a 2-qubit circuit (no entanglement) and read counts.


In [ ]:
# Build a 2-qubit circuit and measure.
qc2 = QuantumCircuit(2, 2)
qc2.h(0)
qc2.measure([0, 1], [0, 1])

# Run on local sampler.
result2 = sampler.run([qc2], shots=2048).result()
quasi2 = result2.quasi_dists[0]
counts2 = {k: int(v * 2048) for k, v in quasi2.items()}
print("Counts:", counts2)
plot_histogram(counts2)


## EXERCISE
Apply X on qubit 1 and predict the output distribution.


## GOAL
Submit a Bell state on a real IBM QPU using SamplerV2.


In [ ]:
# Import runtime tools and transpiler.
from qiskit_ibm_runtime import SamplerV2, Session
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Select a real backend.
backend = service.least_busy(simulator=False, operational=True)
print("Using backend:", backend.name)

# Build a Bell circuit and measure.
bell = QuantumCircuit(2, 2)
bell.h(0)
bell.cx(0, 1)
bell.measure([0, 1], [0, 1])

# Transpile for the selected backend.
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
bell_t = pm.run(bell)

# Run on hardware with SamplerV2.
with Session(service=service, backend=backend) as session:
    sampler2 = SamplerV2(session=session)
    job = sampler2.run([bell_t], shots=1024)
    print("Job ID:", job.job_id())
    print("Hardware queues are normal. Continue with other cells while waiting.")
    result_h = job.result()

# Convert quasi distribution to approximate counts.
quasi_h = result_h.quasi_dists[0]
counts_h = {k: int(v * 1024) for k, v in quasi_h.items()}
print("QPU counts (approx):", counts_h)
plot_histogram(counts_h)


## EXERCISE
Change optimization_level (0, 1, 2, 3) and compare circuit depth.


## Common mistakes
- Token missing: set QISKIT_IBM_TOKEN and restart the kernel.
- Wrong interpreter: select the .venv interpreter in VS Code.
- QPU queues: delays are normal. Keep learning while the job runs.
